In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 50)

In [45]:
base_df = pd.read_csv("../data/poverty-rate.csv")
base_df.head()

C:\Users\kevin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7,10,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ind_id,ind_definition,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,Poverty,TotalPop,NumPov,percent,LL_95CI_percent,UL_95CI_percent,percent_SE,percent_RSE,place_decile,CA_RR,ConcentratedCT,version
0,754,"Overall, concentrated, and child (0 to 18 year...",2000,1,AIAN,CA,6,California,NaN,NaN,NaN,NaN,Child,49112.0,12376.0,25.2,NaN,NaN,NaN,NaN,NaN,1.326316,NaN,01/22/2014 12:04:00 PM
1,754,"Overall, concentrated, and child (0 to 18 year...",2000,2,Asian,CA,6,California,NaN,NaN,NaN,NaN,Child,855747.0,136920.0,16.0,NaN,NaN,NaN,NaN,NaN,0.842105,NaN,01/22/2014 12:04:00 PM
2,754,"Overall, concentrated, and child (0 to 18 year...",2000,3,AfricanAm,CA,6,California,NaN,NaN,NaN,NaN,Child,653820.0,196146.0,30.0,NaN,NaN,NaN,NaN,NaN,1.578947,NaN,01/22/2014 12:04:00 PM
3,754,"Overall, concentrated, and child (0 to 18 year...",2000,4,Latino,CA,6,California,NaN,NaN,NaN,NaN,Child,4050825.0,1085621.0,26.8,NaN,NaN,NaN,NaN,NaN,1.410526,NaN,01/22/2014 12:04:00 PM
4,754,"Overall, concentrated, and child (0 to 18 year...",2000,5,NHOPI,CA,6,California,NaN,NaN,NaN,NaN,Child,31806.0,6425.0,20.2,NaN,NaN,NaN,NaN,NaN,1.063158,NaN,01/22/2014 12:04:00 PM


In [50]:
base_df = base_df.loc[base_df['geoname'] == 'California']

In [51]:
cols = base_df.columns.tolist()

In [52]:
for i in range(len(cols)):
    print(i, cols[i])

0 ind_id
1 ind_definition
2 reportyear
3 race_eth_code
4 race_eth_name
5 geotype
6 geotypevalue
7 geoname
8 county_name
9 county_fips
10 region_name
11 region_code
12 Poverty
13 TotalPop
14 NumPov
15 percent
16 LL_95CI_percent
17 UL_95CI_percent
18 percent_SE
19 percent_RSE
20 place_decile
21 CA_RR
22 ConcentratedCT
23 version


In [53]:
base_df = base_df.drop(base_df.iloc[:, [0, 1, 3, 5, 6, 7, 8, 9, 10, 11, 16, 17, 18, 19, 20, 21, 22, 23]], axis=1)

In [54]:
base_df.head()

,reportyear,race_eth_name,Poverty,TotalPop,NumPov,percent
0,2000,AIAN,Child,49112.0,12376.0,25.2
1,2000,Asian,Child,855747.0,136920.0,16.0
2,2000,AfricanAm,Child,653820.0,196146.0,30.0
3,2000,Latino,Child,4050825.0,1085621.0,26.8
4,2000,NHOPI,Child,31806.0,6425.0,20.2


In [55]:
base_df.columns = ['Year', 'Race', 'Type', 'Population', 'Poverty', 'Percent']
base_df = base_df.fillna(value=0)

In [56]:
races = {'AIAN': 'American Indian and/or Alaska Native',
        'Asian': 'Asian', 'White': 'White', 'Other': 'Other',
        'AfricanAm': 'Black or African American', 
         'NHOPI': 'Native Hawaiian and Other Pacific Islander',
        'Latino': 'Hispanic', 'Multiple': 'Mixed Race', 'Total':'Total'}

In [57]:
grouped_df = base_df.groupby(['Year', 'Race', 'Type'], as_index=False).sum()
grouped_df['Percent'] = round(grouped_df['Poverty']*100/grouped_df['Population'], 1)
grouped_df = grouped_df.dropna()
grouped_df.head()

,Year,Race,Type,Population,Poverty,Percent
0,2000,AIAN,Child,49112.0,12376.0,25.2
1,2000,AIAN,Concentrated,36692.0,0.0,0.0
2,2000,AIAN,Overall,178984.0,36692.0,20.5
3,2000,AfricanAm,Child,653820.0,196146.0,30.0
4,2000,AfricanAm,Concentrated,488751.0,63625.0,13.0


In [58]:
for i in range(len(grouped_df)):
    race = grouped_df.iloc[i, 1]
    grouped_df.iloc[i, 1] = races[race]

grouped_df.head()

,Year,Race,Type,Population,Poverty,Percent
0,2000,American Indian and/or Alaska Native,Child,49112.0,12376.0,25.2
1,2000,American Indian and/or Alaska Native,Concentrated,36692.0,0.0,0.0
2,2000,American Indian and/or Alaska Native,Overall,178984.0,36692.0,20.5
3,2000,Black or African American,Child,653820.0,196146.0,30.0
4,2000,Black or African American,Concentrated,488751.0,63625.0,13.0


In [59]:
grouped_df = grouped_df.loc[grouped_df['Race'] != 'Total']

In [60]:
race_df = grouped_df.loc[grouped_df['Type'] == 'Overall'].groupby("Race", as_index=False).sum()
race_df['Percent'] = round(race_df['Poverty']*100/race_df['Population'], 1)
race_df.to_csv("../data/race/total.csv", index=False)
race_df

,Race,Population,Poverty,Percent
0,American Indian and/or Alaska Native,332414.0,67071.0,20.2
1,Asian,8332688.0,958856.0,11.5
2,Black or African American,4345881.0,930198.0,21.4
3,Hispanic,24422713.0,5101384.0,20.9
4,Mixed Race,1735304.0,228389.0,13.2
5,Native Hawaiian and Other Pacific Islander,235241.0,32310.0,13.7
6,Other,180865.0,26570.0,14.7
7,White,30923832.0,2487594.0,8.0


In [61]:
type_df = grouped_df.groupby("Type", as_index=False).sum()
type_df['Percent'] = round(type_df['Poverty']*100/type_df['Population'], 1)
type_df.to_csv("../data/type/total.csv", index=False)
type_df.head()

,Type,Population,Poverty,Percent
0,Child,18575158.0,3544841.0,19.1
1,Concentrated,9832372.0,747682.0,7.6
2,Overall,70508938.0,9832372.0,13.9


In [62]:
year_df = grouped_df.loc[grouped_df['Type'] == 'Overall'].groupby("Year", as_index=False).sum()
# year_df = grouped_df.groupby("Year", as_index=False).sum()
# year_df['Population'] = grouped_df.loc[grouped_df['Type'] == 'Overall'].groupby("Year", as_index=False).sum()['Population']
year_df['Percent'] = round(year_df['Poverty']*100/year_df['Population'], 1)
year_df.to_csv("../data/byYear/total.csv", index=False)
year_df

,Year,Population,Poverty,Percent
0,2000,33871648.0,4809155.0,14.2
1,2006-2010,36637290.0,5023217.0,13.7


In [63]:
race_year = grouped_df.groupby(['Race', 'Year'], as_index=False).sum()
race_year['Population'] = grouped_df.loc[grouped_df['Type'] == 'Overall'].\
                            groupby(['Race', 'Year'], as_index=False).sum()['Population']
race_year['Percent'] = round(race_year['Poverty']*100/race_year['Population'], 1)
race_year.to_csv('../data/byYear/race.csv', index=False)
race_year.head()

,Race,Year,Population,Poverty,Percent
0,American Indian and/or Alaska Native,2000,178984.0,49068.0,27.4
1,American Indian and/or Alaska Native,2006-2010,153430.0,38952.0,25.4
2,Asian,2000,3648860.0,635630.0,17.4
3,Asian,2006-2010,4683828.0,614889.0,13.1
4,Black or African American,2000,2181926.0,748522.0,34.3


In [64]:
type_year = grouped_df.groupby(['Type', 'Year'], as_index=False).sum()
type_year['Percent'] = round(type_year['Poverty']*100/type_year['Population'], 1)
type_year.to_csv('../data/byYear/type.csv', index=False)
type_year

,Type,Year,Population,Poverty,Percent
0,Child,2000,9249829.0,1762955.0,19.1
1,Child,2006-2010,9325329.0,1781886.0,19.1
2,Concentrated,2000,4809155.0,428349.0,8.9
3,Concentrated,2006-2010,5023217.0,319333.0,6.4
4,Overall,2000,33871648.0,4809155.0,14.2
5,Overall,2006-2010,36637290.0,5023217.0,13.7


In [65]:
race_type = grouped_df.groupby(['Race', 'Type'], as_index=False).sum()
race_type['Percent'] = round(race_type['Poverty']*100/race_type['Population'], 1)
race_type.to_csv('../data/race/type.csv', index=False)